In [2]:
!rm -rf self-expanding-nets

In [3]:
!git clone https://github.com/CTLab-ITMO/self-expanding-nets -b multi-layer-replace

Cloning into 'self-expanding-nets'...


In [3]:
# %pip list

In [4]:
%pip uninstall senmodel
%pip install -U -e ./self-expanding-nets/ 

Found existing installation: senmodel 1.0.0
Uninstalling senmodel-1.0.0:
  Successfully uninstalled senmodel-1.0.0
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/jupyter/work/resources/self-expanding-nets
  Checking if build backend supports build_editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for senmodel (pyproject.toml) ... done
  Created wheel for senmodel: filename=senmodel-1.0.0-0.editable-py3-none-any.whl size=2589 sha256=31a49d565255b0b18cf16f6b932f8cb0a5e85d98c85ddae946b15cf3d0ef68f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-xujz59c7/wheels/8c/6a/b2/f49550010ceefc8bea4532e456bf143d5e692219c03f491bf5
Successfully built senmodel

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
# pip install --force-reinstall numpy==1.23.5

In [1]:
import wandb


In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import time

In [4]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [5]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # transforms.Lambda(lambda x: x.view(-1))
    ]
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse',
           'ship', 'truck')

from torch.utils.data import random_split

full_train_dataset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)

test_dataset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

train_dataset, val_dataset = random_split(full_train_dataset, [0.8, 0.2])


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

next(iter(train_loader))[0].shape

Files already downloaded and verified
Files already downloaded and verified


torch.Size([64, 3, 32, 32])

In [7]:
# class SimpleFCN(nn.Module):
#     def __init__(self, input_size=3 * 32 * 32, hidden_size=16):
#         super(SimpleFCN, self).__init__()
#         self.fc0 = nn.Linear(input_size, 10)
#         # self.fc1 = nn.Linear(hidden_size, 10)
#         self.act = nn.ReLU()

#     def forward(self, x):
#         x = self.fc0(x)
#         return x

In [8]:
class ExpandingHead(nn.Module):
    def __init__(self, num_classes=10):
        super(ExpandingHead, self).__init__()
        
        backbone = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
        in_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    
    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

In [29]:
model = ExpandingHead()
model.freeze_backbone()

Using cache found in /tmp/xdg_cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [30]:
model( torch.randn(2, 3, 32, 32))

tensor([[ 0.1823,  0.0016,  0.6423,  0.3690, -0.6165,  0.1254,  0.7410, -0.0591,
          0.0706,  0.0746],
        [ 0.1767,  0.1642,  0.2257,  0.4217, -0.4804,  0.6781,  0.4599, -0.0932,
          0.0022,  0.3109]], grad_fn=<AddmmBackward0>)

In [31]:

sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc], device=device)

In [32]:
hyperparams = {
    "num_epochs": 64,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc": 0.6}, # 0.0 -> all edges, 1.0 -> no edges
    "threshold": 0.005,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 5e-5,
    "delete_after": 2,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc': 0.6}, threshold: 0.005, min_delta_epoch_replace: 8, window_size: 5, lr: 5e-05, delete_after: 2, task_type: classification, fully_connected: False, max_to_replace: 900"

In [33]:
wandb.finish()

run = wandb.init(
    project="sen-cifar-pretr-backbone",
    name="garbage",
    config=hyperparams
)


wandb: uploading output.log; uploading wandb-summary.json
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               acc amount ▁███████████▇▇▇▇▇▇▇▇▇▇▇▆▆
wandb:               len_choose ▁█
wandb:                       lr ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: n_params over train_time █▁▁▁▁▁▁▁▁▁▁▁▃▂▁▁▁▁▁▁▁▁▁▄▂
wandb:            params amount ▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄▄▄▄▄▄▄▄▄██
wandb:             params ratio ▆▆▇▅▃█▆▇█▃█▃▅▇▇▇▅▁▅▇▄▃▂▅█
wandb: params to replace amount ▂▃▂▄▅▁▃▂▁▅▁▅▄▂▂▂▄█▄▂▅▆▇▅▁
wandb:               train loss █▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train time ▁▅▅▅▅▅▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇▇▇▇█
wandb: train_time over n_params ▁▇▇▇▇▇▇▇▇▇█▇▅▇████▇█▇█▇▅▆
wandb:             val accuracy ▁████████████████████████
wandb:                 val loss █▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:               acc amount 0.00129
wandb:               len_choose 5
wandb:                       lr 5e-05
wandb: n_param

In [34]:

criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams, device)

100%|██████████| 625/625 [00:10<00:00, 57.34it/s]


Epoch 1/64, Train Loss: 2.1607, Val Loss: 1.8392, Val Accuracy: 0.5387


100%|██████████| 625/625 [00:14<00:00, 43.87it/s]


Epoch 2/64, Train Loss: 0.8890, Val Loss: 0.4566, Val Accuracy: 0.9792


100%|██████████| 625/625 [00:14<00:00, 44.04it/s]


...

Epoch 32/64, Train Loss: 0.0024, Val Loss: 0.0761, Val Accuracy: 0.9766


100%|██████████| 625/625 [00:15<00:00, 40.17it/s]


Epoch 33/64, Train Loss: 0.0027, Val Loss: 0.0710, Val Accuracy: 0.9780


 32%|███▏      | 202/625 [00:05<00:10, 40.28it/s]


KeyboardInterrupt: 